In [0]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from sklearn.utils import shuffle
import pickle
import os
import matplotlib.pyplot as plt

In [1]:
from google.colab import  drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/Colab Notebooks/hr-human

/content/drive/My Drive/Colab Notebooks/hr-human


In [0]:
mkdir data

In [0]:
link= 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip'

In [0]:
!wget --no-check-certificate \https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \-O ./horse-or-human.zip

--2020-06-03 14:12:46--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘./horse-or-human.zip’

./horse-or-human.zi 100%[===================>] 142.65M  48.9MB/s    in 2.9s    

2020-06-03 14:12:49 (48.9 MB/s) - ‘./horse-or-human.zip’ saved [149574867/149574867]



In [0]:
data_folder = './data'

In [0]:
!unzip -q horse-or-human.zip -d $data_folder

In [0]:
train_folder ='/content/drive/My Drive/Colab Notebooks/hr-human/data'

In [5]:
ls

data/  horse-or-human.zip


thiết lập model


In [0]:
model   = Sequential() 

In [0]:
model.add(Conv2D(8,kernel_size=(3,3),strides=(1,1),activation='relu',input_shape= [150,150,3],kernel_regularizer='l2'))

In [0]:
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(8,kernel_size=(3,3),strides=(1,1),activation='relu',kernel_regularizer='l2' ))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 8)         584       
Total params: 808
Trainable params: 808
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.add(MaxPooling2D((2,2)))

In [0]:
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu',kernel_regularizer='l2' ))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 8)         584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        2336      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9248)              0

In [0]:
model.compile(loss='binary_crossentropy',optimizer='SGD',metrics=['acc'])

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
train_generator = ImageDataGenerator(rescale= 1/255.0)

In [15]:
generated_train_data = train_generator.flow_from_directory(
    train_folder,
    target_size = (150,150), # nen
    batch_size = 128,
    class_mode ='binary'

)

Found 1027 images belonging to 2 classes.


In [16]:
generated_train_data.class_indices

{'horses': 0, 'humans': 1}

In [0]:
history = model.fit_generator(generated_train_data,
                              steps_per_epoch = 8,
                              epochs=10,verbose=1,
                              )